# Lecture 11 Helicity Formalism
The example in lecture 11
based on [Lecture 11](https://indico.ific.uv.es/event/6803/contributions/21223) by Vincent Mathieu 

In [ ]:
%pip install -q gdown matplotlib numpy

In [ ]:
import warnings

import gdown
import numpy as np
from IPython.display import display

warnings.filterwarnings("ignore")

In [ ]:
# https://indico.ific.uv.es/event/6803/contributions/21223/attachments/11221/15563/Three-particles-3.dat
filename = gdown.cached_download(
    url="https://indico.ific.uv.es/event/6803/contributions/21223/attachments/11221/15563/Three-particles-3.dat",
    # path="data/Three-particles-3.dat",
    # md5="a49ebfd97ae6a02023291df665ab924c",
    quiet=True,
    verify=False,
)
data = np.loadtxt(filename)
data.shape

In [ ]:
data

In [ ]:
# data[0::4]

In [ ]:
n_final_state = 3
pa, p1, p2, p3 = (data[i::4].T for i in range(n_final_state + 1))
p0 = p1 + p2 + p3
pb = p0 - pa

In [ ]:
def mass(p: np.ndarray) -> np.ndarray:
    return np.sqrt(mass_squared(p))


def mass_squared(p: np.ndarray) -> np.ndarray:
    return p[0] ** 2 - np.sum(p[1:] ** 2, axis=0)

In [ ]:
m0 = mass(p0)
print(f"{m0.mean():.4g} +/- {m0.std():.4g}")

In [ ]:
from IPython.display import Math

display(Math(Rf"m_a = {mass(pa).mean():.3g}\text{{ GeV}}"))
display(Math(Rf"m_b = {mass(pb).mean():.3g}\text{{ GeV}}"))
for i, p in enumerate([p0, p1, p2, p3]):
    display(Math(Rf"m_{i} = {mass(p).mean():.3g}\text{{ GeV}}"))

In [ ]:
from particle import Particle


def find_candidates(
    mass: float, delta: float = 0.001, charge: float | None = None
) -> list[Particle]:
    def identify(p) -> bool:
        if p.pdgid in {21}:
            return False
        if charge is not None and p.charge != charge:
            return False
        if (mass - delta) < 1e-3 * p.mass < (mass + delta):
            return True
        return False

    return Particle.findall(identify)


ma = mass(pa).mean()
mb = mass(pb).mean()
m1 = mass(p1).mean()
m2 = mass(p2).mean()
m3 = mass(p3).mean()
initial_state = (
    find_candidates(ma.mean(), delta=1e-4)[0],
    find_candidates(mb.mean())[0],
)
final_state = tuple(find_candidates(m.mean())[0] for m in [m1, m2, m3])
display(
    Math(R"\text{Incoming: }" + ", ".join(f"{p.latex_name}" for p in initial_state)),
    Math(R"\text{Outgoing: }" + ", ".join(f"{p.latex_name}" for p in final_state)),
)

a photon&nbsp;$\gamma$ hitting a proton&nbsp;$p$ and producing a meson&nbsp;$\eta$, pion&nbsp;$\pi^0$, and proton&nbsp;$p$.

In [ ]:
s12 = mass_squared(p1 + p2)
s23 = mass_squared(p2 + p3)
s31 = mass_squared(p3 + p1)

m12 = mass(p1 + p2)
m23 = mass(p2 + p3)
m31 = mass(p3 + p1)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
fig.suptitle("Dalitz plot – 2D histogram")
ax.hist2d(s12, s23, bins=100, cmin=1)
ax.set_xlabel(R"$s_{12}\;\left[\mathrm{GeV}^2\right]$")
ax.set_ylabel(R"$s_{23}\;\left[\mathrm{GeV}^2\right]$")
# fig.tight_layout()
plt.show()